## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-13-00-00 +0000,wsj,PrizePicks Sells Majority Stake to Lottery Ope...,https://www.wsj.com/business/media/prizepicks-...
1,2025-09-22-13-00-00 +0000,wsj,Atlanta Fed President Raphael Bostic said infl...,https://www.wsj.com/economy/central-banking/fe...
2,2025-09-22-13-00-00 +0000,wsj,A full transcript of WSJ’s Nick Timiraos’s int...,https://www.wsj.com/economy/central-banking/tr...
3,2025-09-22-13-00-00 +0000,missionlocal,Feline affordable housing,https://missionlocal.org/2025/09/sf-photos-fel...
4,2025-09-22-12-58-59 +0000,nyt,Dominican Republic Says It Recovered Cocaine F...,https://www.nytimes.com/2025/09/22/us/dominica...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
183,trump,41
67,kirk,33
66,charlie,27
186,memorial,14
48,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
100,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...,138
146,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,136
153,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...,134
138,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,132
24,2025-09-22-11-42-53 +0000,bbc,Trump hails Charlie Kirk as martyr to thousand...,https://www.bbc.com/news/articles/ckgee0x9p40o...,131


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
100,138,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...
216,52,2025-09-21-13-36-12 +0000,nypost,"Canada, UK and Australia all recognize Palesti...",https://nypost.com/2025/09/21/world-news/canad...
147,50,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
109,40,2025-09-22-03-21-19 +0000,nypost,New dad receives 24-year-old letter penned by ...,https://nypost.com/2025/09/21/us-news/new-dad-...
153,33,2025-09-21-21-50-25 +0000,nypost,"President Trump, Elon Musk shake hands at Char...",https://nypost.com/2025/09/21/us-news/presiden...
131,30,2025-09-22-00-09-00 +0000,wsj,"In a powerful moment, Charlie Kirk’s wife, Eri...",https://www.wsj.com/politics/charlie-kirk-memo...
215,25,2025-09-21-13-38-54 +0000,nypost,Trump presses Bondi to prosecute his enemies i...,https://nypost.com/2025/09/21/us-news/trump-pr...
173,24,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
170,23,2025-09-21-20-11-42 +0000,nypost,Zohran Mamdani gives Rudy Giuliani some credit...,https://nypost.com/2025/09/21/us-news/zohran-m...
168,22,2025-09-21-20-13-11 +0000,latimes,LAPD will increase patrols around Jewish synag...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
